In [1]:
def setLogging(level,file=None):
    """
    This method set the logging level and the format of the logs
    """
    
    ## setting logging level
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG if level == 'debug' else logging.INFO)

    ## setting log format for print
    handlerStream = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handlerStream.setFormatter(formatter)
    logger.addHandler(handlerStream)
    
    ## setting log format for save
    if file is not None:
        handlerFile = logging.FileHandler(file, mode='w')
#         formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        handlerFile.setFormatter(formatter)
        logger.addHandler(handlerFile)

In [2]:
## required packages
import numpy as np
# import matplotlib.pyplot as plt
# import os, time
import importlib
import im.fcn_im as fcn_im
import pandas as pd
# from numpy.random import standard_normal
# from scipy.linalg import cholesky
import model, fcn_gen, os, time, warnings
from scipy import sparse

## setting logging level (e.g. DEBUG or INFO)
import logging
# from setLogging import *
setLogging('info')

In [3]:
## define number of realizations and generate samples
flag_sample_exist = True
nsamp_im = 4
nsamp_pgd = 4
flag_corr_d = False
flag_corr_T = False
if flag_sample_exist:
    logging.info(f"Samples exist: {nsamp_im} IM samples - will not perform sampling.")
else:
    logging.info(f"Number of IM samples = {nsamp_im}; spatial correlation = {flag_corr_d} and cross-correlation = {flag_corr_T}.")
logging.info(f"Number of PGD samples = {nsamp_pgd} (for lateral spreading and settlement only).")

## base inputs
base_dir = r'C:\Users\barry\Desktop\CEC\OpenSRA_local\10000yr_allsites' # base directory with input files and sub-directories
site_loc_file  = os.path.join(base_dir,'CA_pipe_midpt_allsites.txt')
im_tool = 'sha'
gm_dir = os.path.join(base_dir,'gm_10000yr_allsites_ask14_10samp')
im_dir = os.path.join(base_dir,'gm_10000yr_allsites_ask14_'+str(nsamp_im)+'samp')
# im_dir = os.path.join(base_dir,'gm_10000yr_allsites_ask14_'+str(nsamp_im)+'samp_pgv_const_sigma')
dm_dir = os.path.join(base_dir,'dm_resource','damages','repair_rates_'+str(nsamp_im)+'samp')
rup_meta_file = os.path.join(base_dir,'rup_meta_10000yr.hdf5')
l_seg_file = os.path.join(base_dir,'seg_length_allsites.txt')
logging.info(f"Defined base directories and paths to required input files.")
logging.info(f"Damage outputs will be stored in {dm_dir}.")

## number of groups
groups2run = np.loadtxt(os.path.join(base_dir,'groups2run_allsites.txt'),dtype=str)
rup_per_group = 100
multi_max = int(np.ceil(len(groups2run)/rup_per_group))
logging.info(f"The number of groups to run = {len(groups2run)}.")

2020-06-30 19:46:35,030 - root - INFO - Samples exist: 4 IM samples - will not perform sampling.
2020-06-30 19:46:35,030 - root - INFO - Number of PGD samples = 4 (for lateral spreading and settlement only).
2020-06-30 19:46:35,041 - root - INFO - Defined base directories and paths to required input files.
2020-06-30 19:46:35,042 - root - INFO - Damage outputs will be stored in C:\Users\barry\Desktop\CEC\OpenSRA_local\10000yr_allsites\dm_resource\damages\repair_rates_4samp.
2020-06-30 19:46:35,050 - root - INFO - The number of groups to run = 1977.


In [12]:
## reload model.py if it has been modified
importlib.reload(model)
logging.info(f'Load/reloaded "model.py".')

## create class object
A = model.assessment()
logging.info(f'Created model class object named "A".')

## cases to run
dmgs = ['rr_pgd'] # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# dmg_procs = ['hazus','ala','orourke'] # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
dmg_procs = ['orourke'] # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
dmds = ['land'] # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
ims = ['pga','pgv'] # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

## for sampling
# dmgs = [] # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# dmg_procs = [] # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# dmds = []
# ims = ['pgv']

if 'rr_pgv' in dmgs and len(dmgs) == 1:
    dmds = []
    ims = ['pgv']
logging.info(f"Damages include: {dmgs}. Demands include: {dmds if len(dmds) > 0 else None}.")

## decide if samples are needed
flag_gen_sample = False
if flag_sample_exist:
    flag_gen_sample = True
flag_get_IM = False # default to False
if 'rr_pgv' in dmgs:
    flag_get_IM = True
if 'rr_pgd' in dmgs:
    if 'hazus' in dmg_procs:
        flag_get_IM = True
    else:
        if 'ls' in dmds or 'land' in dmds or 'surf' in dmds:
            flag_get_IM = True
if flag_get_IM:
    logging.info(f"Samples are will be imported.")
else:
    logging.info(f"Samples are not needed.")

## check if probability of liquefaction is needed
if 'rr_pgd' in dmgs and ('ls' in dmds or 'gs' in dmds):
    flag_p_liq = True
else:
    flag_p_liq = False
logging.info(f"Flag for probability of liquefaction = {flag_p_liq}.")
    
## check if liquefaction susceptibility is needed
if 'rr_pgd' in dmgs and ('ls' in dmds or 'gs' in dmds):
    flag_liq_susc = True
else:
    flag_liq_susc = False
logging.info(f"Flag for liquefaction susceptibility = {flag_liq_susc}.")

## check if probability of landslide is needed
if 'rr_pgd' in dmgs and 'land' in dmds:
    flag_p_land = True
else:
    flag_p_land = False
logging.info(f"Flag for probability of landslide = {flag_p_land}.")

## increment to print output messages
flag_save_dmg = True # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
if len(dmgs) == 0:
	flag_save_dmg = False
if flag_get_IM or flag_gen_sample:
    inc1 = 5 # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
#     inc2 = rup_per_group # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    inc2 = 100 # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    logging.info(f"Will print messages every {inc1} groups (each group contains {rup_per_group} ruptures).")
if flag_save_dmg:
    logging.info(f"Will save damage outputs every {inc2} groups (each group contains {rup_per_group} ruptures).")
else:
    logging.info(f"Damage outputs will not be saved.")

## Load site params depending on demands needed
if flag_liq_susc or flag_p_liq or ('rr_pgd' in dmgs and 'land' in dmds):
    
    ## read inputs for Zhu et al. (2017)
    zhu_inputs = pd.read_csv(os.path.join(base_dir,'CA_Zhu_pipe_midpt_allsites_withWB_v2_remap.csv'))
    
    ## for liquefaction-induced demands
    if flag_liq_susc or flag_p_liq:
        
        ## import general inputs
        wtd = zhu_inputs['WTD_30m (m)'].values
        dr = zhu_inputs['DistRivers (km)'].values
        dc = zhu_inputs['DistCoast (km)'].values
        dw = zhu_inputs['DistAnyWaterNoWB (km)'].values
        precip = zhu_inputs['CA_Precip (mm)'].values
        vs30 = zhu_inputs['vs30_opensha (m/s)'].values
        
        ## change all -9999 entries to NaN
        find_val = -9999
        set_val = np.nan
        wtd = fcn_gen.find_set_nan(wtd,find_val,set_val)
        dr = fcn_gen.find_set_nan(dr,find_val,set_val)
        dc = fcn_gen.find_set_nan(dc,find_val,set_val)
        dw = fcn_gen.find_set_nan(dw,find_val,set_val)
        precip = fcn_gen.find_set_nan(precip,find_val,set_val)
        vs30 = fcn_gen.find_set_nan(vs30,find_val,set_val)
        
        ## elevation from DEM maps
        z = np.ones(vs30.shape)*10 ## set to 10 for now, get data from DEM map later

        ## for ground settlement
        if 'ls' in dmds:
            dwWB = zhu_inputs['DistAnyWaterWB (km)'].values
            dwWB = fcn_gen.find_set_nan(dwWB,find_val,set_val)
            dwWB = dwWB*1000 # convert to meters

        logging.info(f"Imported site parameters for liquefaction-induced demands.")
        
    ## for landslide-induced demands
    if 'land' in dmds:
        ky = zhu_inputs['ky_inf_bray'].values
#         ky[ky==0] = 0.0001
        logging.info(f"Imported site parameters for landslide-induced demands.")
    
if 'surf' in dmds:
    intersect_dir = os.path.join(base_dir,'gm_resource','rup_intersect')
    logging.info(f"Defined directory with fault crossings: {intersect_dir}.")

2020-06-30 19:56:39,416 - root - INFO - Load/reloaded "model.py".
2020-06-30 19:56:39,432 - root - INFO - Created model class object named "A".
2020-06-30 19:56:39,432 - root - INFO - Damages include: ['rr_pgd']. Demands include: ['land'].
2020-06-30 19:56:39,433 - root - INFO - Samples are will be imported.
2020-06-30 19:56:39,435 - root - INFO - Flag for probability of liquefaction = False.
2020-06-30 19:56:39,436 - root - INFO - Flag for liquefaction susceptibility = False.
2020-06-30 19:56:39,437 - root - INFO - Flag for probability of landslide = True.
2020-06-30 19:56:39,438 - root - INFO - Will print messages every 5 groups (each group contains 100 ruptures).
2020-06-30 19:56:39,439 - root - INFO - Will save damage outputs every 100 groups (each group contains 100 ruptures).
2020-06-30 19:56:39,888 - root - INFO - Imported site parameters for landslide-induced demands.


In [13]:
## define multiplier; grops to run = multiplier * rup_per_group
multi_start = 0
n_multi = 1 # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
if flag_get_IM or flag_gen_sample:
    multi_end = multi_start+n_multi
    logging.info(f"Multiplers to run: {multi_start} to {multi_end-1}.")
else:
    multi_end = multi_start+1
    logging.info(f"Samples not needed, no need to loop multi.")

time_start = time.time()

## loop through multipliers
for multi in range(multi_start,multi_end):
    
    ## define range of ruptures for IM
    range_start = rup_per_group*multi
    if flag_get_IM or flag_gen_sample:
#         range_end = range_start+rup_per_group
        range_end = range_start+5
        logging.debug(f"Multi {multi}...")
    else:
        range_end = range_start+1
    count = range_start

    ## loop through groups
    for rup_group in groups2run[range_start:range_end]:
        count += 1
        count_EDP = 0
        logging.info(f'Count = {count}: rupture group = {rup_group}...')
        
        #####################################################################
        #####################################################################
        logging.debug(f"-------------Intensity Measures-------------")
        #####################################################################
        ## load GM predictions and create random variable
        A.create_RV(im_tool, gm_dir, ims, rup_meta_file, site_loc_file, l_seg_file=l_seg_file, 
                    flag_clear_dict=True, flag_sample_exist=flag_sample_exist, rup_group=rup_group)
        logging.debug(f'\tIM_rv: Updated "_RV_dict".')
        
        #####################################################################
        ## generate/import samples
        if flag_get_IM or flag_gen_sample:
            path_sample = os.path.join(im_dir,rup_group)
            flag_sample_with_sigma_total = True
            sigma_aleatory = 0.400
            n_decimals = 3
            A.get_IM(nsamp_im, ims, flag_corr_d, flag_corr_T, 
                     flag_sample_with_sigma_total=flag_sample_with_sigma_total,
                     sigma_aleatory=sigma_aleatory,
                     flag_clear_dict=True, flag_sample_exist=flag_sample_exist,
                     path_sample=path_sample)
            if flag_sample_exist:
                logging.debug(f'\tIM_sim: Updated "_IM_dict" using path to samples = {path_sample}.')
            else:
                if not os.path.isdir(os.path.join(im_dir,rup_group)):
                    os.mkdir(os.path.join(im_dir,rup_group))
                for im in ims:
                    for samp_i in range(nsamp_im):
                        sparse_mat = A._IM_dict[im][samp_i].log1p().tocsc()
                        sparse_mat.data = np.round(sparse_mat.data,decimals=n_decimals)
                        sparse.save_npz(os.path.join(im_dir,rup_group,'pgv_samp_'+str(samp_i)+'.npz'),sparse_mat)
                logging.debug(f'\tIM_sim: Updated "_IM_dict" by sampling.')

        #####################################################################
        #####################################################################
        logging.debug(f"-------------Engineering Demand Parameters-------------")
        #####################################################################
        if flag_p_liq or flag_liq_susc:
            count_EDP += 1
            ## use Zhu et al. (2017)
            category = 'liq'
            method = 'zhu_etal_2017'
            return_param = []
            if flag_p_liq:
                return_param.append('p_liq')
            if flag_liq_susc:
                return_param.append('liq_susc')
            logging.debug(f'\tReturn params = {return_param}.')
            flag_pgv = True
            flag_M = True
            dc_cutoff = 20
            flag_clear_dict = True if count_EDP == 1 else False
#             logging.debug(f"LIQ: count_EDP = {count_EDP}, flag_clear_dict = {flag_clear_dict}")
            logging.debug(f'\tEDP_liq: Calculate "{return_param}" using "{method}".')
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                A.get_EDP(category=category, method=method, return_param=return_param,
                          flag_clear_dict=flag_clear_dict, flag_pgv=flag_pgv, flag_M=flag_M, 
                          vs30=vs30, precip=precip, dc=dc, dr=dr, dw=dw, 
                          wtd=wtd, dc_cutoff=dc_cutoff, nsamp_im=nsamp_im)

        #####################################################################
        if 'ls' in dmds:
            count_EDP += 1
            ## get pgd_ls from Grant et al. (2016), same as HAZUS (FEMA, 2014)
            category = 'ls'
            method = 'hazus_2014_ls'
            return_param = ['pgd_ls']
            source_dict = ['_EDP_dict']
            source_param = ['liq_susc']
            source_method = ['zhu_etal_2017']
            flag_pga = True
            flag_M = True
            dw_cutoff = 50
            flag_clear_dict = True if count_EDP == 1 else False
            eps_aleatory = np.ones(nsamp_pgd)
            wgt_aleatory = np.ones(nsamp_pgd)/nsamp_pgd
            eps_epistemic = [-1,1]
            # eps_epistemic = [0]
#             logging.debug(f"LS: count_EDP = {count_EDP}, flag_clear_dict = {flag_clear_dict}")
            logging.debug(f'\tEDP_ls: Calculate "{return_param}" using "{method}" and "{source_param}" from "{source_method}".')
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                A.get_EDP(category=category, method=method, return_param=return_param,
                          flag_clear_dict=flag_clear_dict, flag_pga=flag_pga, flag_M=flag_M, 
                          source_dict=source_dict, source_param=source_param, source_method=source_method, 
                          dw=dwWB, z=z, nsamp_im=nsamp_im, flag_extrap_Epgd=True, dw_cutoff=dw_cutoff,
                          eps_aleatory=eps_aleatory, wgt_aleatory=wgt_aleatory, eps_epistemic=eps_epistemic)
         
        #####################################################################   
        if 'gs' in dmds:
            count_EDP += 1
            ## get pgd_gs from HAZUS (FEMA, 2014)
            category = 'gs'
            method = 'hazus_2014_gs'
            return_param = ['pgd_gs']
            source_dict = ['_EDP_dict']
            source_param = ['liq_susc']
            source_method = ['zhu_etal_2017']
            flag_clear_dict = True if count_EDP == 1 else False
            eps_aleatory = np.ones(nsamp_pgd)
            wgt_aleatory = np.ones(nsamp_pgd)/nsamp_pgd
            eps_epistemic = [-1,1]
            # eps_epistemic = [0]
#             logging.debug(f"GS: count_EDP = {count_EDP}, flag_clear_dict = {flag_clear_dict}")
            logging.debug(f'\tEDP_gs: Calculate "{return_param}" using "{method}" and "{source_param}" from "{source_method}".')
            A.get_EDP(category=category, method=method, return_param=return_param, 
                      flag_clear_dict=flag_clear_dict, 
                      source_dict=source_dict, source_param=source_param, source_method=source_method,
                      eps_aleatory=eps_aleatory, wgt_aleatory=wgt_aleatory, eps_epistemic=eps_epistemic)
            
        #####################################################################              
        if 'land' in dmds:
            count_EDP += 1
            ## get pgd_ls from Grant et al. (2016), same as HAZUS (FEMA, 2014)
            category = 'land'
            method = 'bray_macedo_2019'
            return_param = ['pgd_land']
            if flag_p_land:
                return_param.append('p_land')
            logging.debug(f'\tReturn params = {return_param}.')
            gm_type = 'general'
            flag_pgv = True
            flag_pga = True
            flag_M = True
            flag_clear_dict = True if count_EDP == 1 else False
            eps_aleatory = [-1.65,0,1.65]
            wgt_aleatory = [0.2,0.6,0.2]
            eps_epistemic = [-1.65,0,1.65]
#             eps_epistemic = [999]
#             logging.debug(f"LAND: count_EDP = {count_EDP}, flag_clear_dict = {flag_clear_dict}")
            logging.debug(f'\tEDP_land: Calculate "{return_param}" using "{method}".')
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                A.get_EDP(category=category, method=method, return_param=return_param,
                          flag_clear_dict=flag_clear_dict, flag_pga=flag_pga, flag_M=flag_M, flag_pgv=flag_pgv,
                          ky=ky, nsamp_im=nsamp_im, gm_type=gm_type,
                          eps_aleatory=eps_aleatory, wgt_aleatory=wgt_aleatory, eps_epistemic=eps_epistemic)
                        
        #####################################################################  
        if 'surf' in dmds:
            count_EDP += 1
            ## set up for surface fault rupture
            rows = []
            cols = []
            dim_rup = len(A._RV_dict['rup']['src'])
            dim_d = len(A._rup_site_dict['site_lon'])
            count_seg = 0
            for src in A._RV_dict['rup']['src']:
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    seg_list = np.loadtxt(os.path.join(intersect_dir,'src_'+str(src)+'.txt'),dtype=int,ndmin=1)
                for seg in seg_list:
                    rows.append(count_seg)
                    cols.append(seg)
                count_seg += 1
            rows = np.asarray(rows)
            cols = np.asarray(cols)
            mat = sparse.coo_matrix((np.ones(len(rows)),(rows,cols)),shape=(dim_rup,dim_d))
            logging.debug(f'\tEDP_surf: Generated matrix of flags for fault crossing.')
            
            #####################################################################
            ## pgd_surf from Wells and coppersmith (1994), with Steve Thompson's coefficients
            category = 'surf'
            method = 'wells_coppersmith_1994'
            return_param = ['pgd_surf']
            flag_M = True
            flag_clear_dict = True if count_EDP == 1 else False
            eps_aleatory = [-1.65,0,1.65]
            wgt_aleatory = [0.2,0.6,0.2]
#             eps_epistemic = [999,-1.65,0,1.65]
            eps_epistemic = [999]
#             logging.debug(f"SURF: count_EDP = {count_EDP}, flag_clear_dict = {flag_clear_dict}")
            logging.debug(f'\tEDP_surf: Calculate "{return_param}" using "{method}".')
            logging.debug(f"epsilons for aleatory variability = {eps_aleatory}, epsilons for epistemic uncertainty = {eps_epistemic} (if 999, then use lumped sigma).")
            A.get_EDP(category=category, method=method, return_param=return_param, 
                      flag_clear_dict=flag_clear_dict, flag_M=flag_M, mat_seg2calc=mat,
                      eps_aleatory=eps_aleatory, wgt_aleatory=wgt_aleatory, eps_epistemic=eps_epistemic)

        #####################################################################
        #####################################################################
        logging.debug(f"-------------Damage Measures-------------")
        #####################################################################
        category = 'rr'
        if 'rr_pgv' in dmgs:
            return_param = ['rr_pgv']
            flag_pgv = True
            eps_epistemic = [-1.65,0,1.65]
            sigma_epistemic = 0.65
            for proc_i in dmg_procs:
                ## set damage label
                if 'hazus' in proc_i:
                    method = proc_i+'_2014'+'_rr'
                    flag_orourke_pgv = False
                elif 'ala' in proc_i:
                    method = proc_i+'_2001'+'_rr'
                    flag_orourke_pgv = False
                elif 'orourke' in proc_i:
                    method = proc_i+'_2020'+'_rr'
                    flag_orourke_pgv = True
                else:
                    method = None
                    logging.info("procedure does not exist...exiting")
                
                ## set method name and run calculation
                logging.debug(f'\tDM_rr_pgv: Calculate "{return_param}" using "{method}".')
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    A.get_DM(category, method, return_param, ims,
                             flag_pgv=flag_pgv, nsamp_im=nsamp_im, flag_orourke_pgv=flag_orourke_pgv,
							 eps_epistemic=eps_epistemic, sigma_epistemic=sigma_epistemic)
        
        #####################################################################
        if 'rr_pgd' in dmgs:
            return_param = ['rr_pgd']
            flag_rup_depend = True
            flag_proc_repeat = False
            for proc_i in dmg_procs:
                ## set damage label
                if 'hazus' in proc_i:
                    method = proc_i+'_2014'+'_rr'
                    pgd_cutoff = 0
                elif 'ala' in proc_i:
                    method = proc_i+'_2001'+'_rr'
                    pgd_cutoff = 0
                elif 'orourke' in proc_i:
                    method = proc_i+'_2020'+'_rr'
                    pgd_cutoff = 4*2.54
                else:
                    method = None
                    logging.info("procedure does not exist...exiting")

                ## set method name and loop through all pgd demands
                for dmd_i in dmds:
                    ## initialize lists
                    source_dict = ['_EDP_dict']
                    source_param = []
                    source_method = []
                    ##
                    pgd_label = 'pgd_'+dmd_i
                    source_param.append(pgd_label)
                    if 'ls' in dmd_i or 'gs' in dmd_i:
                        source_method.append('hazus_2014_'+dmd_i)
                        source_dict.append('_EDP_dict')
                        source_param.append('p_liq')
                        source_method.append('zhu_etal_2017')
                    elif 'land' in dmd_i:
                        source_method.append('bray_macedo_2019')
                        source_dict.append('_EDP_dict')
                        source_param.append('p_land')
                        source_method.append('bray_macedo_2019')
                    elif 'surf' in dmd_i:
                        source_method.append('wells_coppersmith_1994')
                    logging.debug(f'\tDM_rr_pgd: Calculate "{return_param}" using "{method}" and "{source_param}" from "{source_method}".')
                    with warnings.catch_warnings():
                        warnings.simplefilter("ignore")
                        A.get_DM(category=category, method=method, return_param=return_param, 
                                 flag_rup_depend=flag_rup_depend,
                                 source_dict=source_dict, source_param=source_param, source_method=source_method, 
                                 nsamp_im=nsamp_im, pgd_cutoff=pgd_cutoff, pgd_label=pgd_label)

    
        #####################################################################
        #####################################################################
        if (flag_get_IM or flag_gen_sample) and count % inc1 == 0:
                logging.info(f"-------------After {count*rup_per_group} groups: {np.round(time.time()-time_start,decimals=2)} secs-------------")
                time_start = time.time()
        
        #####################################################################
        #####################################################################
#         if flag_save_dmg:
#             if count % inc2 == 0 or count == len(groups2run):
#                 logging.info(f"\tSaving damages...")
#                 for dmg_i in A._DM_dict.keys():
#                     count_proc = 0
#                     for proc_i in A._DM_dict[dmg_i].keys():
#                         if 'rr_pgd' in dmg_i:
#                             for i in A._DM_dict[dmg_i][proc_i]['source_param']:
#                                 if 'pgd' in i:
#                                     dmd_i = i
#                             str_dmd_i = dmd_i[dmd_i.find('_')+1:]+'_'
#                         else:
#                             str_dmd_i = ''
#                         str_proc_i = A._DM_dict[dmg_i][proc_i]['method'][0:A._DM_dict[dmg_i][proc_i]['method'].find('_')]
#                         if A._DM_dict[dmg_i][proc_i]['eps_epistemic'] == 999:
#                             str_eps = 'epiTotal_'
#                         else:
#                             str_eps = 'epi'+str(A._DM_dict[dmg_i][proc_i]['eps_epistemic'])+'_'
#                             str_eps = str_eps.replace('.','o')
#                             str_eps = str_eps.replace('-','m')
#                         str_range = 'rup_'+str(range_start)+'_'+str(range_end-1)
#                         save_name = dmg_i+'_'+str_proc_i+'_'+str_dmd_i+str_eps+str_range+'.npz'
#                         save_path = os.path.join(dm_dir,save_name)
#                         logging.info(f"\t\t{save_name}")
#                         sparse.save_npz(save_path,np.transpose(A._DM_dict[dmg_i][proc_i]['output']))
#                         count_proc += 1

logging.info(f"...done with current set")

2020-06-30 19:56:39,951 - root - INFO - Multiplers to run: 0 to 0.
2020-06-30 19:56:39,953 - root - INFO - Count = 1: rupture group = 0_99...


[0.00168772 0.0003293  0.00030903 0.00334794 0.00477603 0.01578869
 0.00040647 0.00047842 0.00054019 0.00034039]
[0.27890749 0.20114947 0.19861019 0.31985671 0.34341037 0.43618347
 0.20980005 0.21675094 0.22208022 0.20248573]
c 0 0.2
[0.00262069 0.00010676 0.00092995 0.00039751 0.0057703  0.00593302
 0.00069035 0.001047   0.00072222 0.00439111]
[0.028252   0.00414048 0.01517316 0.00911188 0.0453648  0.04612809
 0.01268938 0.0162918  0.01303767 0.03850742]
b 1 0.6
[0.00348686 0.00035787 0.00130031 0.00082924 0.00030012 0.00371589
 0.00836958 0.00168442 0.00850022 0.00219977]
[0.32246817 0.20452482 0.26473368 0.24195557 0.19745101 0.32659732
 0.38418533 0.27879842 0.3853772  0.29408686]
b 2 0.2
[0.00309662 0.0002844  0.00113799 0.00067603 0.00249498 0.00725331
 0.00122543 0.00739017 0.00083924 0.0019279 ]
[0.31490363 0.19533739 0.25776724 0.23227006 0.30158753 0.37334234
 0.26161206 0.37474075 0.24253648 0.28642886]
c 0 0.2
[0.00014006 0.00040106 0.00400447 0.0033612  0.00183869 0.005910

2020-06-30 19:56:53,375 - root - INFO - Count = 2: rupture group = 100000_100099...


[1.02744564e-04 2.76337352e-05 3.15643208e-04 4.85614802e-04
 7.57625406e-05 3.76523536e-05 1.84231898e-04 3.32309195e-05
 5.23744807e-03 5.79560226e-05]
[0.15934989 0.12254283 0.19945241 0.21739942 0.14993082 0.13036422
 0.17909067 0.12714769 0.34980334 0.14210839]
b 2 0.2
[0.00184387 0.00026499]
[0.28388749 0.19259474]
b 0 0.2
[0.00101898 0.00062955 0.00109079 0.0028381  0.00066696 0.00045081
 0.00052217]
[0.01602875 0.0120066  0.01669724 0.02963582 0.01242962 0.00982643
 0.01073222]
b 1 0.6
[0.0006308  0.00069585 0.00186743 0.00088815 0.00073341 0.00091568
 0.00099713 0.0002693  0.00155516 0.00178582]
[0.22907567 0.23361636 0.28460901 0.24529992 0.2360856  0.24680186
 0.25104435 0.19321828 0.27438146 0.28207676]
b 2 0.2
[0.00028426 0.00154619 0.00042275 0.00058052 0.00084673 0.00135382
 0.0013859  0.00080282 0.00334942 0.00101545]
[0.19531833 0.27406446 0.21145423 0.22530128 0.24296787 0.26687768
 0.2681305  0.24039387 0.31988509 0.25196012]
b 0 0.2
[0.00059016 0.01055608 0.00095713

2020-06-30 19:57:04,263 - root - INFO - Count = 3: rupture group = 10000_10099...


...

[]
[]
b 2 0.2
[0.00018169 0.00058275 0.00069812 0.00054761 0.00041933 0.00021266
 0.00047837 0.00069802 0.00137498 0.00076005]
[0.17859468 0.22547365 0.23376856 0.22268665 0.21111125 0.18430449
 0.21674693 0.23376177 0.26770676 0.23777642]
b 0 0.2
[0.00379763 0.00036202 0.00064951 0.00066025 0.00421499 0.00088133
 0.00031411 0.00046065 0.00027969 0.00056689]
[0.03529454 0.00861475 0.01223351 0.01235455 0.0375732  0.01469209
 0.00791129 0.0099546  0.00737922 0.01127458]
b 1 0.6
[0.00105834 0.00138744 0.00546344 0.00072286 0.00311747 0.00011609
 0.00046542 0.0003235  0.00091211 0.00093806]
[0.25405358 0.26819009 0.35277134 0.23540222 0.3153264  0.16329027
 0.21555997 0.20043521 0.24660936 0.2479967 ]
b 2 0.2
[0.00079103 0.00474481 0.00059204 0.00036415 0.0007973  0.00081738
 0.00026559 0.00029216 0.00557681 0.00103745]
[0.23968391 0.34296016 0.2261885  0.20523696 0.24006224 0.24125957
 0.1926823  0.19639213 0.35422329 0.25304243]
b 0 0.2
[0.00132997 0.00132924 0.00117319 0.00307813 0.002

2020-06-30 19:57:22,571 - root - INFO - Count = 5: rupture group = 100100_100199...


[3.39198133e-04 1.53303103e-04 3.09806719e-04 1.25720325e-04
 2.61513219e-03 3.15882794e-04 4.38929380e-03 3.06773501e-04
 3.58890110e-03 5.61926328e-05]
[0.20234417 0.17262758 0.19870929 0.16591326 0.30443791 0.19948268
 0.33765947 0.19831866 0.32433387 0.1412329 ]
b 2 0.2
[]
[]
b 0 0.2
[]
[]
b 1 0.6
[0.0006297  0.00100735]
[0.22899557 0.25155687]
b 2 0.2
[]
[]
b 0 0.2
[0.00680165 0.00088446 0.00158676]
[0.05006895 0.01472341 0.0209077 ]
b 1 0.6
[1.33283968e-03 2.36715912e-04 4.40771564e-04 1.26476117e-02
 1.36469047e-03 1.61169705e-04 6.94730118e-05 1.25474082e-03
 1.19891603e-03 4.94411107e-04]
[0.2660453  0.18829791 0.21322725 0.41725515 0.26730485 0.17436394
 0.14735444 0.26285171 0.26047003 0.21818136]
b 2 0.2
[0.00091258 0.00026073 0.01045077 0.00109376 0.00058192 0.00037536
 0.00200934 0.00062327]
[0.24663446 0.19197267 0.40163326 0.25573149 0.22541008 0.20648603
 0.28880905 0.22852554]
b 0 0.2
[0.00171777 0.00044863 0.00058365 0.01522097 0.00189156 0.00038006
 0.00050614 0.000

2020-06-30 19:57:34,021 - root - INFO - -------------After 500 groups: 54.07 secs-------------
2020-06-30 19:57:34,021 - root - INFO - ...done with current set


[0.00048714 0.00057236 0.0013696  0.00075572 0.00234931 0.00035753
 0.00033864 0.00026482 0.00067695 0.0001168 ]
[0.21753619 0.22466392 0.26749707 0.23750491 0.2979806  0.20448533
 0.20227718 0.19257115 0.23233332 0.16348934]
b 2 0.2


In [6]:
A._DM_dict

{'rr_pgd': {'method1': {'method': 'orourke_2020_rr',
   'source_param': ['pgd_land', 'p_land'],
   'source_method': ['bray_macedo_2019', 'bray_macedo_2019'],
   'eps_epistemic': -1.65,
   'eps_aleatory': [-1.65, 0, 1.65],
   'wgt_aleatory': [0.2, 0.6, 0.2],
   'output': <4x123316 sparse matrix of type '<class 'numpy.float64'>'
   	with 395 stored elements in Compressed Sparse Column format>},
  'method2': {'method': 'orourke_2020_rr',
   'source_param': ['pgd_land', 'p_land'],
   'source_method': ['bray_macedo_2019', 'bray_macedo_2019'],
   'eps_epistemic': 0,
   'eps_aleatory': [-1.65, 0, 1.65],
   'wgt_aleatory': [0.2, 0.6, 0.2],
   'output': <4x123316 sparse matrix of type '<class 'numpy.float64'>'
   	with 2037 stored elements in Compressed Sparse Column format>},
  'method3': {'method': 'orourke_2020_rr',
   'source_param': ['pgd_land', 'p_land'],
   'source_method': ['bray_macedo_2019', 'bray_macedo_2019'],
   'eps_epistemic': 1.65,
   'eps_aleatory': [-1.65, 0, 1.65],
   'wgt_al

In [13]:
A._DM_dict['rr_pgd']['method1']['output'].data[:20]

array([0.02621234, 0.00236126, 0.01045677, 0.02151469, 0.0194128 ,
       0.06489118, 0.91677073, 0.01890516, 0.00099298, 0.04694919,
       0.02364384, 0.01459185, 0.02677483, 0.02508768, 0.01740504,
       0.01997873, 0.01782076, 0.01424091, 0.03779048, 0.04947491])

In [11]:
A._DM_dict

{'rr_pgd': {'method1': {'method': 'orourke_2020_rr',
   'source_param': ['pgd_land', 'p_land'],
   'source_method': ['bray_macedo_2019', 'bray_macedo_2019'],
   'eps_epistemic': -1.65,
   'eps_aleatory': [-1.65, 0, 1.65],
   'wgt_aleatory': [0.2, 0.6, 0.2],
   'output': <4x123316 sparse matrix of type '<class 'numpy.float64'>'
   	with 0 stored elements in Compressed Sparse Column format>},
  'method2': {'method': 'orourke_2020_rr',
   'source_param': ['pgd_land', 'p_land'],
   'source_method': ['bray_macedo_2019', 'bray_macedo_2019'],
   'eps_epistemic': 0,
   'eps_aleatory': [-1.65, 0, 1.65],
   'wgt_aleatory': [0.2, 0.6, 0.2],
   'output': <4x123316 sparse matrix of type '<class 'numpy.float64'>'
   	with 0 stored elements in Compressed Sparse Column format>},
  'method3': {'method': 'orourke_2020_rr',
   'source_param': ['pgd_land', 'p_land'],
   'source_method': ['bray_macedo_2019', 'bray_macedo_2019'],
   'eps_epistemic': 1.65,
   'eps_aleatory': [-1.65, 0, 1.65],
   'wgt_aleator

In [11]:
aa.data[:20]

array([0.02621234, 0.00236126, 0.01045677, 0.02151469, 0.0194128 ,
       0.06489118, 0.91677073, 0.01890516, 0.00099298, 0.04694919,
       0.02364384, 0.01459185, 0.02677483, 0.02508768, 0.01740504,
       0.01997873, 0.01782076, 0.01424091, 0.03779048, 0.04947491])

In [17]:
A._IM_dict['pgv'][0].data[:100]

array([ 12.12770151,  12.43036281, 131.7129635 ,  76.58710194,
       149.11434034, 145.39642656, 107.96625098,  81.06869367,
        97.66998668, 135.75300673,  62.3896885 ,  99.69343478,
        87.85745566,  66.42027574,  13.08470397,  13.58802266,
        13.69269546,  17.58731004,   9.7577716 ,   8.4343066 ,
        13.64256039,  30.31218441,  12.08779752,  19.20081549,
        16.56396997,  29.28259403,  11.88209644,  14.08729928,
         6.97421375,   7.87159413,  28.98274397,   9.2686643 ,
         5.74758219,  17.23264027,  12.62489099,  14.04128261,
         8.51650215,  18.13716207,  14.12790005,   7.95590811,
        19.46592814,   6.52120622,  14.22050185,  13.36736222,
         8.18214274,  10.3286225 ,  10.26105406,  11.40353888,
         9.47924263,  13.79080563,  10.15816901,   8.27031727,
         6.50262419,   7.67758614,   9.01232005,   3.85098018,
         9.61153064,  11.30484652,   6.42667086,  16.61646662,
         7.98977733,  17.85906474,  12.8290343 ,   7.33

In [7]:
aa = sparse.load_npz(r'C:\Users\barry\Desktop\CEC\OpenSRA_local\10000yr_allsites\gm_10000yr_allsites_ask14_4samp\0_99\pga_samp_0.npz')

In [8]:
aa.data[:500]

array([0.046, 0.019, 0.193, 0.024, 0.047, 0.108, 0.074, 0.126, 0.332,
       0.154, 0.078, 0.092, 0.071, 0.345, 0.048, 0.16 , 0.016, 0.079,
       0.111, 0.068, 0.069, 0.307, 0.255, 0.048, 0.099, 0.029, 0.152,
       0.033, 0.137, 0.014, 0.062, 0.121, 0.108, 0.113, 0.16 , 0.191,
       0.242, 0.133, 0.024, 0.18 , 0.012, 0.207, 0.043, 0.049, 0.077,
       0.055, 0.144, 0.423, 0.172, 0.129, 0.045, 0.022, 0.12 , 0.034,
       0.059, 0.026, 0.039, 0.068, 0.061, 0.228, 0.193, 0.199, 0.113,
       0.033, 0.094, 0.054, 0.01 , 0.141, 0.023, 0.026, 0.028, 0.059,
       0.128, 0.254, 0.722, 0.054, 0.032, 0.017, 0.146, 0.016, 0.143,
       0.025, 0.051, 0.125, 0.039, 0.129, 0.29 , 0.191, 0.096, 0.028,
       0.027, 0.065, 0.046, 0.11 , 0.006, 0.049, 0.114, 0.082, 0.107,
       0.103, 0.959, 0.192, 0.029, 0.018, 0.102, 0.079, 0.035, 0.027,
       0.034, 0.05 , 0.052, 0.297, 0.18 , 0.322, 0.072, 0.047, 0.024,
       0.022, 0.022, 0.099, 0.017, 0.015, 0.04 , 0.042, 0.097, 0.419,
       0.199, 0.144,

In [19]:
A._IM_dict['pgv'][0].data[:10]

array([2.04953069e+04, 1.54446468e+01, 7.68537563e+08, 4.64179097e+01,
       1.17943031e+01, 1.32775386e+06, 3.92510770e+13, 7.94921576e+05,
       1.12692035e+06, 1.62144747e+08])

In [18]:
A._DM_dict['rr_pgv']['method1']['output'].data[:20]

array([4.07982657e+24, 6.90461764e+26, 2.87230796e+21, 7.15360111e+21,
       3.83480013e+27, 4.75959319e+16, 6.26775562e+08, 8.36034010e+40,
       7.50372644e+18, 6.36109778e+20, 3.36055224e+27, 4.84709747e+13,
       7.89586718e+36, 1.37944308e+32, 3.98535079e+23, 1.01316168e+16,
       2.30312051e+31, 2.53310714e+31, 9.63145089e+22, 1.05727860e+17])

In [24]:
max(A._IM_dict['pgv'][0].data)

2.3752658197334042e+222

In [35]:
baseDir = r'C:\Users\barry\Desktop\CEC\OpenSRA_local\10000yr_allsites\gm_10000yr_allsites_ask14_4samp_pgv_const_sigma'
listDir = os.listdir(baseDir)
n_decimals = 3

In [37]:
for dir_i in listDir:
# dir_i = listDir[0]
    listFile = []
    listFile = os.listdir(os.path.join(baseDir,dir_i))
    for file in listFile:
    # file = listFile[0]
#         sparse_mat = sparse.load_npz(os.path.join(baseDir,dir_i,file)).log1p()
#         sparse_mat.data = np.round(sparse_mat.data,decimals=n_decimals)
#         sparse.save_npz(os.path.join(baseDir,dir_i,file),sparse_mat)
        #     sparse_mat = A._IM_dict[im][samp_i].log1p().tocsc()
        #     sparse_mat.data = np.round(sparse_mat.data,decimals=n_decimals)
        #     sparse.save_npz(os.path.join(im_dir,rup_group,'pgv_samp_'+str(samp_i)+'.npz'),sparse_mat)

In [34]:
sparse_mat.data[:100]

array([2.391, 1.335, 3.066, 1.581, 1.267, 2.715, 3.475, 2.68 , 2.704,
       2.991, 2.561, 1.707, 1.192, 2.015, 0.897, 2.835, 1.21 , 1.821,
       3.571, 3.018, 3.318, 2.349, 3.079, 2.835, 1.94 , 1.554, 2.309,
       1.409, 2.426, 1.557, 1.964, 2.77 , 2.584, 2.208, 3.093, 3.315,
       2.633, 1.613, 1.754, 3.049, 1.207, 3.036, 1.467, 1.395, 2.468,
       1.811, 1.936, 2.535, 3.787, 2.188, 1.666, 2.145, 3.275, 1.593,
       2.205, 1.462, 1.95 , 2.981, 2.534, 3.329, 2.947, 3.664, 2.847,
       2.143, 1.128, 2.371, 0.928, 2.321, 1.67 , 2.07 , 1.801, 3.34 ,
       2.332, 2.588, 2.51 , 2.104, 1.319, 1.703, 2.391, 0.976, 3.008,
       1.454, 1.367, 2.692, 2.899, 2.474, 2.226, 2.933, 2.493, 1.902,
       1.901, 2.883, 0.934, 2.667, 1.387, 1.725, 2.704, 2.295, 2.766,
       2.827])

In [24]:
aa = sparse.load_npz(r'C:\Users\barry\Desktop\CEC\OpenSRA_local\10000yr_allsites\gm_10000yr_allsites_ask14_4samp\0_99\pgv_samp_0.npz')

In [32]:
aa

<100x123316 sparse matrix of type '<class 'numpy.float64'>'
	with 1460753 stored elements in Compressed Sparse Column format>

In [25]:
# aa = aa.expm1()
aa.data[:100]

array([2.709, 1.162, 3.228, 0.998, 1.947, 2.518, 2.425, 3.267, 3.678,
       3.051, 2.716, 2.096, 2.47 , 3.066, 1.581, 2.92 , 1.24 , 2.752,
       2.66 , 2.223, 2.975, 3.492, 2.735, 2.189, 1.621, 1.537, 4.077,
       1.854, 3.069, 0.72 , 2.943, 3.136, 2.51 , 3.638, 2.541, 2.779,
       3.415, 2.788, 1.959, 3.393, 1.195, 3.13 , 2.598, 2.112, 2.208,
       1.926, 3.285, 4.089, 3.909, 3.246, 2.185, 1.08 , 2.754, 1.38 ,
       2.58 , 1.661, 1.789, 2.201, 1.499, 2.496, 3.996, 3.331, 3.252,
       1.81 , 2.183, 2.841, 0.923, 2.699, 1.031, 2.483, 1.233, 1.17 ,
       2.008, 3.044, 3.075, 2.759, 1.409, 1.115, 2.684, 1.233, 2.855,
       1.709, 2.687, 3.09 , 1.7  , 2.776, 4.659, 3.297, 3.017, 2.354,
       1.379, 2.42 , 1.564, 2.53 , 1.39 , 2.48 , 2.422, 1.729, 2.435,
       2.978])